In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import sparse

# Function to augment data with noise
def augment_data(X, noise_factor=0.1):
    noise = np.random.randn(*X.shape) * noise_factor
    return X + noise

# Activation functions
def leaky_relu(Z, alpha=0.01):
    return np.where(Z > 0, Z, alpha * Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

# Initialize weights
def initialize_weights(layers):
    np.random.seed(42)
    weights = []
    for i in range(1, len(layers)):
        weight = np.random.randn(layers[i-1], layers[i]) * np.sqrt(2 / layers[i-1])  # He initialization
        bias = np.zeros((1, layers[i]))
        weights.append((weight, bias))
    return weights

# Forward propagation
def forward_propagation(X, weights):
    A = X
    caches = []
    for weight, bias in weights[:-1]:
        Z = np.dot(A, weight) + bias
        A = leaky_relu(Z)
        caches.append((A, Z))
    Z_final = np.dot(A, weights[-1][0]) + weights[-1][1]
    A_final = sigmoid(Z_final)
    caches.append((A_final, Z_final))
    return A_final, caches

# Backward propagation
def backward_propagation(X_batch, y_batch, caches, weights, learning_rate, lambda_reg):
    m = X_batch.shape[0]
    A_final, _ = caches[-1]
    y_batch = np.array(y_batch).reshape(-1, 1)
    dZ_final = A_final - y_batch
    dW_final = np.dot(caches[-2][0].T, dZ_final) / m + lambda_reg * weights[-1][0]
    db_final = np.sum(dZ_final, axis=0, keepdims=True) / m
    weights[-1] = (weights[-1][0] - learning_rate * dW_final, weights[-1][1] - learning_rate * db_final)

    dA = dZ_final
    for i in reversed(range(len(weights) - 1)):
        A_prev, Z_prev = caches[i]
        dZ = dA.dot(weights[i + 1][0].T) * (Z_prev > 0)
        dW = np.dot(caches[i - 1][0].T, dZ) / m if i > 0 else np.dot(X_batch.T, dZ) / m
        db = np.sum(dZ, axis=0, keepdims=True) / m
        weights[i] = (weights[i][0] - learning_rate * dW, weights[i][1] - learning_rate * db)
        dA = dZ
    return weights

# Training function
def train(X_train, y_train, layers, epochs, batch_size, learning_rate, lambda_reg):
    weights = initialize_weights(layers)
    m = X_train.shape[0]
    for epoch in range(epochs):
        for i in range(0, m, batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            y_pred, caches = forward_propagation(X_batch, weights)
            weights = backward_propagation(X_batch, y_batch, caches, weights, learning_rate, lambda_reg)
    return weights

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred_binary):
    cm = confusion_matrix(y_true, y_pred_binary)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal", "Attack"])
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()

# Load dataset and preprocessing
data = pd.read_csv(r'P:\project ides for final ptoject\project\IDS_Federated learning model\model\cleaned_Network_dataset.csv', low_memory=False)
columns_to_keep = [
    'duration', 'proto', 'service', 'src_bytes', 'dst_bytes', 'src_pkts', 'dst_pkts',
    'src_ip_bytes', 'dst_ip_bytes', 'conn_state', 'label', 'type'
]
data_filtered = data[columns_to_keep]
data_filtered['type'] = data_filtered['type'].apply(lambda x: 0 if x == 'normal' else 1)
X = data_filtered.drop(columns=['type'])
y = data_filtered['type']
categorical_columns = ['proto', 'service', 'conn_state']
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_columns),
    ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)
])
X = preprocessor.fit_transform(X)
if sparse.issparse(X):
    X = X.toarray()

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
X_train_augmented = augment_data(X_train)

# Model training
layers = [X_train.shape[1], 32, 16, 1]
weights = train(X_train_augmented, y_train, layers, epochs=15, batch_size=128, learning_rate=0.00001, lambda_reg=0.01)

# Model evaluation
y_test_pred, _ = forward_propagation(X_test, weights)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_test_pred_binary)
precision = precision_score(y_test, y_test_pred_binary)
recall = recall_score(y_test, y_test_pred_binary)
f1 = f1_score(y_test, y_test_pred_binary)

# Print metrics
print("\nTest Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot confusion matrix
plot_confusion_matrix(y_test, y_test_pred_binary)


In [ ]:
user_input= {
    'duration': 0.000557,           
    'proto': 'udp',        
    'service': 'dns',       
    'src_bytes': 0,         
    'dst_bytes': 298,        
    'src_pkts': 0,         
    'dst_pkts': 2,          
    'src_ip_bytes': 0,    
    'dst_ip_bytes': 354,       
    'conn_state': 'SHR',       
    'label': None              
}
prediction = predict_user_input(user_input, loaded_weights, preprocessor)
print(f"Predicted class for DoS input: {'Attack' if prediction[0][0] == 1 else 'Normal'}")
